In [4]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2022
mmdd_str = today.strftime('%m%d')
mmdd_str

'0418'

In [2]:
#today = date(2022, 2, 25)
mmdd_str = today.strftime('%m%d')
mmdd_str

'0418'

In [10]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  2


In [11]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  POMPUI
Year:  2022
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335742270
Published:  Mon, 18 Apr 2022 20:02:24 +0700

----------------------------------

F45: 1
Title:  TISCO
Year:  2022
Quarter:  1
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335714940
Published:  Mon, 18 Apr 2022 13:01:28 +0700


In [12]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']]

,name,year,quarter,published
0,POMPUI,2022,y,"Mon, 18 Apr 2022 20:02:24 +0700"
1,TISCO,2022,1,"Mon, 18 Apr 2022 13:01:28 +0700"


In [13]:
df.loc[df.name == 'POMPUI','year'] = 2021
df.groupby(['year','quarter']).count()

,,name,link,published
year,quarter,,,
2021,y,1,1,1
2022,1,1,1,1


In [15]:
df.loc[df.quarter == 'y','quarter'] = 4
df.groupby(['year','quarter']).count()

,,name,link,published
year,quarter,,,
2021,4,1,1,1
2022,1,1,1,1


In [7]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == 'e','quarter'] = 4
#df.replace({"quarter" : "1"}, "3", inplace=True)
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,36,36,36,36


In [8]:
df.loc[df.quarter == '1','quarter'] = 4
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,36,36,36,36


In [9]:
df.loc[df.quarter == ' ','quarter'] = 4
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,36,36,36,36


In [10]:
df.loc[df.quarter == 'S','quarter'] = 4
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,36,36,36,36


In [11]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == '1','year'] = 2022
#df.replace({"quarter" : "1"}, "3", inplace=True)
df.groupby(['year']).count()

,name,quarter,link,published
year,,,,
2021,36,36,36,36


In [16]:
df.shape

(2, 5)

In [17]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df.shape

(2, 5)

In [18]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(one_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [19]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

299

In [20]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(301, 7)

### Tickers that won't be input

In [21]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
]
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link


In [22]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

0

In [23]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,POMPUI,2021.0,4,"Mon, 18 Apr 2022 20:02:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335742270
1,TISCO,2022.0,1,"Mon, 18 Apr 2022 13:01:28 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335714940


In [24]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

2

In [25]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(403, 9)

In [26]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(404, 14)

### There are no ticker records

In [27]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
]

,name,year,quarter,published,link,id,market
0,POMPUI,2021.0,4,"Mon, 18 Apr 2022 20:02:24 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335742270,NaN,NaN


In [28]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
].shape

(1, 7)

### There are ticker records

In [29]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market
1,TISCO,2022.0,1,"Mon, 18 Apr 2022 13:01:28 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335714940,531.0,SET50 / SETHD / SETTHSI


In [30]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2

,name,year,quarter,published,link,id,market
1,TISCO,2022.0,1,"Mon, 18 Apr 2022 13:01:28 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335714940,531.0,SET50 / SETHD / SETTHSI


In [31]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2

,name,year,quarter,published,link,id,market
1,TISCO,2022,1,"Mon, 18 Apr 2022 13:01:28 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335714940,531.0,SET50 / SETHD / SETTHSI


In [32]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(one_file, header=True, index=False, sep=',')

In [33]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2021'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(1334, 14)

In [34]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(1335, 19)

### Already input, display profit amt & eps to check with new F45

In [35]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge


In [36]:
df_merge3[df_merge3['_merge'] == 'both'].shape

(0, 19)

In [37]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3.shape

(1, 19)

In [38]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
inp3_str = df_inp3.name+' '+df_inp3.year+' '+df_inp3.quarter+' '+df_inp3.link
inp3_str

0    TISCO 2022 1 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16502335714940
dtype: object